## Test in real time

In [1]:
import os
import time

import numpy as np
from numpy import zeros

from matplotlib import pyplot as plt
import cv2

import mediapipe as mp
from tensorflow import keras

- preapering for reading keypoints using mp holistic model

In [2]:
# FUNCTIONS WORKING WITH MP HOLISTIC MODEL

# Initialazing mp holistic for keypoints
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities



# Finding landmarks with model holistic
def mediapipe_detection(image, model):
    
    # creating contrast on img
    lab= cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) # applying CLAHE to L-channel   - feel free to try different values for the limit and grid size
    cl = clahe.apply(l_channel)
    limg = cv2.merge((cl,a,b)) # merge the CLAHE enhanced L-channel with the a and b channel
    image = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR) #onverting image from LAB Color model to BGR color spcae
    
    # detections
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion BGR 2 RGB
    image.flags.writeable = False                  # image is no longer writeable
    results = model.process(image)                 # make prediction
    image.flags.writeable = True                   # image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion RGB 2 BGR
    return image, results



# Drawing landmarks
def draw_styled_landmarks(image, results):
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) # draw pose connections
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) # draw left hand connections
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) # draw right hand connections  

    
    
# Preparing vector of date
def extract_keypoints(results):
    
    if (results.pose_landmarks == None): # if there is no landmarks
        return [], [], [], [], [],
        
    else:
        # face
        x = 0
        y = 0
        for i in range(11):
            if results.pose_landmarks.landmark[i].visibility > 0.95:
                x += results.pose_landmarks.landmark[i].x
                y += results.pose_landmarks.landmark[i].y
            else:
                return [], [], [], [], [],
        face = np.array([x/11, y/11]) # the middle point from all points on face
        
        
        # pose
        pose_left = []
        pose_right = []
        
        if(results.pose_landmarks.landmark[11].visibility > 0.85 and results.pose_landmarks.landmark[12].visibility > 0.85): # middle point between choulders
            pose_left.append(((results.pose_landmarks.landmark[11].x + results.pose_landmarks.landmark[12].x) /2) - face[0])
            pose_left.append(((results.pose_landmarks.landmark[11].y + results.pose_landmarks.landmark[12].y) /2) - face[1])
            pose_right.append(((results.pose_landmarks.landmark[11].x + results.pose_landmarks.landmark[12].x) /2) - face[0])
            pose_right.append(((results.pose_landmarks.landmark[11].y + results.pose_landmarks.landmark[12].y) /2) - face[1])
        else:
            pose_left.append(0)
            pose_left.append(0)
            pose_right.append(0)
            pose_right.append(0)

        for i in range(11, 17): # points from pose - arms and shoulder
            if(results.pose_landmarks.landmark[i].visibility > 0.85):
                if(i % 2 == 0):
                    pose_right.append(results.pose_landmarks.landmark[i].x - face[0])
                    pose_right.append(results.pose_landmarks.landmark[i].y - face[1])
                else:     
                    pose_left.append(results.pose_landmarks.landmark[i].x - face[0])
                    pose_left.append(results.pose_landmarks.landmark[i].y - face[1])
            else:
                if(i % 2 == 0):
                    pose_right.append(0)
                    pose_right.append(0) 
                else:
                    pose_left.append(0)
                    pose_left.append(0)     

        pose_left = np.array(pose_left)
        pose_right = np.array(pose_right)

        
        # right hand
        if (results.right_hand_landmarks == None):
            right_hand = []
        else:
            right_hand = np.array([[-1 * (res.x - face[0]), res.y - face[1]] for res in results.right_hand_landmarks.landmark]).flatten()

            
        # left hand
        if (results.left_hand_landmarks == None):
            left_hand = []
        else:
            left_hand = np.array([[res.x - face[0], res.y - face[1]] for res in results.left_hand_landmarks.landmark]).flatten()
   

        return face, pose_left, pose_right, left_hand, right_hand

- preapering video sequence for prediction

In [3]:
# FUNCTIONS FOR extract_sequence

def init_global_variables_on_zero(all_variables = True):
    
    global frame_num, all_keypoints, left_zeros_fixed, right_zeros_fixed
    global number_of_blank_pose_left, number_of_blank_pose_right, number_of_blank_left_hand, number_of_blank_right_hand
    global vector_of_blank_pose_left, vector_of_blank_pose_right, vector_of_blank_left_hand, vector_of_blank_right_hand
    global all_frames_pose_left, all_frames_pose_right, all_frames_left_hand, all_frames_right_hand
    
    
    number_of_blank_pose_left = 0
    number_of_blank_pose_right = 0
    number_of_blank_left_hand = 0
    number_of_blank_right_hand = 0
    
    vector_of_blank_pose_left = []
    vector_of_blank_pose_right = []
    vector_of_blank_left_hand = []
    vector_of_blank_right_hand = []
    
    if all_variables:
        all_frames_pose_left = []
        all_frames_pose_right = []
        all_frames_left_hand = []
        all_frames_right_hand = []
        
        frame_num = 1
        all_keypoints = []
        left_zeros_fixed = False
        right_zeros_fixed = False
        

        
def counting_number_of_blanks():
    
    global frame_num, all_keypoints, left_zeros_fixed, right_zeros_fixed
    global number_of_blank_pose_left, number_of_blank_pose_right, number_of_blank_left_hand, number_of_blank_right_hand
    global vector_of_blank_pose_left, vector_of_blank_pose_right, vector_of_blank_left_hand, vector_of_blank_right_hand
    global all_frames_pose_left, all_frames_pose_right, all_frames_left_hand, all_frames_right_hand
    global pose_left, pose_right, left_hand, right_hand
    
    
    # checking existence of landmarks on body and hands
    if len(left_hand) == 0:
        left_hand = np.zeros(42)
        vector_of_blank_left_hand.append(1)
    else:
        vector_of_blank_left_hand.append(0)
        
    if len(right_hand) == 0:
        right_hand = np.zeros(42)
        vector_of_blank_right_hand.append(1)
    else:
        vector_of_blank_right_hand.append(0)
        
    if len(pose_left) - np.count_nonzero(pose_left) != 0:
        vector_of_blank_pose_left.append(1)
    else:
        vector_of_blank_pose_left.append(0)
            
    if len(pose_right) - np.count_nonzero(pose_right) != 0:
        vector_of_blank_pose_right.append(1)
    else:
        vector_of_blank_pose_right.append(0)
            
    # number of blanks in last 12 frames not including last one
    if (frame_num > sequence_length):
        vector_of_blank_pose_left = vector_of_blank_pose_left[-13:-1]
        vector_of_blank_pose_right = vector_of_blank_pose_right[-13:-1]
        vector_of_blank_left_hand = vector_of_blank_left_hand[-13:-1]
        vector_of_blank_right_hand = vector_of_blank_right_hand[-13:-1]
            
    number_of_blank_pose_left = sequence_length - np.count_nonzero(vector_of_blank_pose_left)
    number_of_blank_pose_right = sequence_length - np.count_nonzero(vector_of_blank_pose_right)
    number_of_blank_left_hand = sequence_length - np.count_nonzero(vector_of_blank_left_hand)
    number_of_blank_right_hand = sequence_length - np.count_nonzero(vector_of_blank_right_hand)
    


# Updating hole story for single part 
def update_all_frames():
    
    global frame_num, all_keypoints, left_zeros_fixed, right_zeros_fixed
    global number_of_blank_pose_left, number_of_blank_pose_right, number_of_blank_left_hand, number_of_blank_right_hand
    global vector_of_blank_pose_left, vector_of_blank_pose_right, vector_of_blank_left_hand, vector_of_blank_right_hand
    global all_frames_pose_left, all_frames_pose_right, all_frames_left_hand, all_frames_right_hand
    global pose_left, pose_right, left_hand, right_hand
    
    
    all_frames_pose_left.append(pose_left)
    all_frames_pose_right.append(pose_right)
    all_frames_left_hand.append(left_hand)
    all_frames_right_hand.append(right_hand)
    
    # extracting last 13 frames
    if (frame_num > sequence_length):
        all_frames_pose_left = all_frames_pose_left[-13:]
        all_frames_pose_right = all_frames_pose_right[-13:]
        all_frames_left_hand = all_frames_left_hand[-13:]
        all_frames_right_hand = all_frames_right_hand[-13:]
    

In [4]:
def fixing_zeros_and_counting_varations(all_frames_part, check_varation = True, fix_zeros = True):  
    
    global frame_num, all_keypoints, left_zeros_fixed, right_zeros_fixed
    global number_of_blank_pose_left, number_of_blank_pose_right, number_of_blank_left_hand, number_of_blank_right_hand
    global all_frames_pose_left, all_frames_pose_right, all_frames_left_hand, all_frames_right_hand
    global pose_left, pose_right, left_hand, right_hand
    
    
    varations = []
    col = []
    # creating vector of colmun
    for it in range(len(all_frames_part[0])):
        for frame in all_frames_part:
            col.append(frame[it])
        
        # fixing zeros in whole 12 frame sequance
        if fix_zeros:
            for it2 in range(1, len(col)-1):
                if(col[it2] == 0 and col[it2 + 1] == 0):
                    # TO DO ... UPgrade global vector_of_blank_pose_left itd na 0 do wartosci it2 na 0, wtedy nie bd musial wchodzic tu nie potrzebnie
                    return [404, 404]
                if(col[it2] == 0):                            
                    col[it2] = (col[it2 - 1] + col[it2 + 1])/2
                    all_frames_part[it2][it] = col[it2]
                    
                    
        #variation from every colmun
        if check_varation:
            # wyliczamy wariancje
            col = 10 * np.array(col[:12]) # *10 working good for next if with treshold
            srednio =  sum(col)/sequence_length 
            varations.append(np.sum( (np.array(col) - srednio)**2 ) / sequence_length )
        col = []
    
    #counting varation and return if check varation
    if check_varation:
        if(np.sum(varations) >= 0.8):
            return [1, 1]
        else:
            return [1, 404]
    
    # return for fixiing zeros
    if(fix_zeros):
        return [1, 404]

In [5]:
# LOGIC

def extract_sequence(results):
    
    global frame_num, all_keypoints, left_zeros_fixed, right_zeros_fixed
    global number_of_blank_pose_left, number_of_blank_pose_right, number_of_blank_left_hand, number_of_blank_right_hand
    global vector_of_blank_pose_left, vector_of_blank_pose_right, vector_of_blank_left_hand, vector_of_blank_right_hand
    global all_frames_pose_left, all_frames_pose_right, all_frames_left_hand, all_frames_right_hand
    global pose_left, pose_right, left_hand, right_hand
       
    
    # extracting keypoints
    face, pose_left, pose_right, left_hand, right_hand = extract_keypoints(results)
        
    if len(face) == 0: # if there is no landmarks
        print("there is nothing on sreen")
        init_global_variables_on_zero()
        return 404
    
    # checking how many blanks is in every single part of body in last 12 frames
    counting_number_of_blanks()  

    # updating whole story for single part
    update_all_frames()
        
    if(frame_num < 13):
        frame_num += 1
        return 404
    
    else:
        
        lh = [404, 404]
        lp = [404, 404]
        ph = [404, 404]
        pp = [404, 404]
        # left hand and arm cant have more then 2 blanks frame and first analizng frame cant contain only zeros
        if(number_of_blank_left_hand <= 2  and number_of_blank_pose_left <= 2 and all_frames_left_hand[0][0] != 0 and len(pose_left)-np.count_nonzero(all_frames_pose_left[0]) == 0):
            # TO DO ...  uzyc flagi i jak mamy dobry wektor 12 klatek to updatowac tylko ostanie zmiany, jak byly 2 z rzedu fla na false by po jakims czasie naprawic caly wektor jak pojawia sie dobre dane
            #if(left_zeros_fixed):
            #    print("cos nowego")
            #else:
            #    print("decyzja")
            lh = fixing_zeros_and_counting_varations(all_frames_left_hand)
            lp = fixing_zeros_and_counting_varations(all_frames_pose_left)

        # right hand and arm cant have more then 2 blanks frame and first analizng frame cant contain only zeros
        if(number_of_blank_right_hand <= 2 and number_of_blank_pose_right <= 2 and  all_frames_right_hand[0][0] != 0 and len(pose_right)-np.count_nonzero(all_frames_pose_right[0]) == 0):
            #if(right_zeros_fixed):
            #    print("cos nowego")
            #else:
            #    print("decyzja")
            ph = fixing_zeros_and_counting_varations(all_frames_right_hand)
            pp = fixing_zeros_and_counting_varations(all_frames_pose_right)
            
        # not good data for prediction too much blanks frame
            
        # not good data for prediction -2 blanks next to eachother or too much blanks frame
        if((lh[0] == 404 or lp[0] == 404) and (ph[0] == 404 or pp[0] == 404)):
            print("coutch not enough landmarks ")
            return 404
           
        # 2 hands are moving
        elif(lh[1] == 1 and ph[1] == 1):
            print("do sieci na 2 rece")  # TO DO ... jak bd sie jebac moge polaczyc pose'y z soba i wyjebac jeden wspolny pnkt#####
            for i in range(sequence_length):
                keypoints = np.concatenate([all_frames_pose_left[i], all_frames_left_hand[i], all_frames_pose_right[i], all_frames_right_hand[i]])                                
                all_keypoints.append(keypoints)
            print()
            return 2   
        
        # 1 hand is moving
        elif(lh[1] == 1 or ph[1] == 1):
            print("do sieci dla 1 reki")
            if(lh[1] == 1):
                for i in range(sequence_length):                  
                    keypoints = np.concatenate([all_frames_pose_left[i], all_frames_left_hand[i]])
                    all_keypoints.append(keypoints)
            else:
                for i in range(sequence_length):
                    keypoints = np.concatenate([all_frames_pose_right[i], all_frames_right_hand[i]])
                    all_keypoints.append(keypoints) 
            print()
            return 1
        
        # no move detected
        else:
            print("do sieci dla nie ruchomych rak")                
            all_keypoints = np.concatenate([sum(all_frames_pose_left)/sequence_length, sum(all_frames_left_hand)/sequence_length, sum(all_frames_pose_right)/sequence_length, sum(all_frames_right_hand)/sequence_length])
            print()
            return 0
        

- importing model and setting dataset properties

In [6]:
# folders info
no_sequences = 50
sequence_length = 12 
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['pierdol_sie', 'hello','I_me','need','thanks', 'drink', 'beer'])
colors = [(255, 153, 51) ,(245,117,16), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245)]

# importing model
model = keras.models.load_model("action2d_e25 _96.h5")

 - vizualitation

In [7]:
# init global variables
init_global_variables_on_zero()
sentence = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.4, min_tracking_confidence=0.4) as holistic:
    while cap.isOpened():

        # read feed
        ret, frame = cap.read()
        
        # make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # draw landmarks
        draw_styled_landmarks(image, results)
        
        # extract sequence
        network_choose= extract_sequence(results)
        
        
        # choosing type of network
        if network_choose == 0:
            print()
            # ------ ------- ------ ------ ------ ------ ------ 
            # TO DO ... siec na nieruchome obrazki
            # ------ ------- ------ ------ ------ ------ ------         
        
        elif network_choose == 1: 
            
            # making prediction
            res = model.predict(np.expand_dims(all_keypoints, axis=0))[0]   
            #print(actions[np.argmax(res)])
            
            # collecting 5 last words for top bar
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # showing probabilities for every word
            for num, prob in enumerate(res):
                cv2.rectangle(image, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
                cv2.putText(image, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
        elif network_choose == 2:
            print()
            # ------ ------- ------ ------ ------ ------ ------ 
            # TO DO ... siec na 2 lapy
            # ------ ------- ------ ------ ------ ------ ------   
        
        # needed because of np concatenation
        all_keypoints = [] 
        
        # showing top bar
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # show to screen
        cv2.imshow('OpenCV ladnmark_test', image)

        # break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not eno

1/1 [==============================] - 0s 28ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 27ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 34ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 29ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 33ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 26ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 29ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 27ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 29ms/step
do sieci dla 1 reki

1/1 [==============================] - 0s 28ms/step
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
coutch not enough landmarks 
c

In [113]:
# siec na 2 lapy
# zrobic dataset na 2 lapy
# zrobic 3 blanki z rzedu to od nowa* zeby zer nie ciagnelo z poczatku jak jakas reka sie pojawi z ukrycia  
# i nie byla wczesniej naprawiana to rozpierdala wariancje jak chcemy sie czegos pozbyc

In [7]:
# uczytelnic kod
# - blocks of already exsiting functions DONE
# - blocks of real time DONE
# - - created func: DONE DONE DONE DONE DONE DONE
# mozliwe ze trzeba zrobic osobny counter blankow dla obu rak, by pozbyc sie znaku zrobionego z poczatkowych 2 zer DONE

# zrobic test data set
# -na jedna reke DONE
# -na 2 rece
# przetestowac rozne sieci dla 2 sieci na jedna i 2 rece
# -na jedna reke
# -na 2 rece

## na przyszlosc
# normalizacja lecimy miedzy klata i twarza na 1 - wszystko co jest potrzebne jest w pobieranej dacie
# mp holistic face false
# dopracowac parametry 
# - wariancja jest w obu a przy tworzeniu ryja moge podciagnac zeby byl tip top i holistic dac duze wymagania zeby cos znalazl w real life na odwrot
# - w pose dla 11 do 17 # to jeszcze do empirycznego ustawienia !!!!!!!!

## training on PC

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [5]:
actions = np.array(['pierdol_sie', 'hello','I_me','need','thanks', 'drink', 'beer'])

In [6]:
label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence+1), "{}.npy".format(frame_num+1)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
        
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [35]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(12,50)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
#model.add(LSTM(128, return_sequences=True, activation='relu'))
#model.add(LSTM(64, return_sequences=True, activation='relu'))
#model.add(LSTM(128, return_sequences=True ,activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=25, callbacks=[tb_callback])
model.summary()

Epoch 1/25
11/11 [==============================] - 4s 33ms/step - loss: 1.9257 - categorical_accuracy: 0.4096
Epoch 2/25
11/11 [==============================] - 0s 29ms/step - loss: 1.7234 - categorical_accuracy: 0.6566
Epoch 3/25
11/11 [==============================] - 0s 29ms/step - loss: 1.6273 - categorical_accuracy: 0.7169
Epoch 4/25
11/11 [==============================] - 0s 30ms/step - loss: 1.0340 - categorical_accuracy: 0.6717
Epoch 5/25
11/11 [==============================] - 0s 29ms/step - loss: 0.6549 - categorical_accuracy: 0.7741
Epoch 6/25
11/11 [==============================] - 0s 29ms/step - loss: 0.5960 - categorical_accuracy: 0.7952
Epoch 7/25
11/11 [==============================] - 0s 28ms/step - loss: 0.8704 - categorical_accuracy: 0.7500
Epoch 8/25
11/11 [==============================] - 0s 29ms/step - loss: 0.7233 - categorical_accuracy: 0.7440
Epoch 9/25
11/11 [==============================] - 0s 29ms/step - loss: 0.6545 - categorical_accuracy: 0.8253
E

In [36]:
model.save('action2d_e25 _96.h5')

In [ ]:
#1 ja         I ME (1 NIE)- https://www.handspeak.com/word/index.php?id=1086
#2 ty         you (1 NIE)-  https://www.handspeak.com/word/index.php?id=2448
!#3 on ono ana !HE (1)-     https://www.handspeak.com/word/index.php?id=1670
#4 potrzobowac Need(2NIE)-  https://www.handspeak.com/word/index.php?id=1471
#5 pomoc      Help(2)-      https://www.handspeak.com/word/index.php?id=1017
#6 chce       Want(2)-      https://www.handspeak.com/word/index.php?id=2347
#7 prosze     Pleas(2)-     https://www.handspeak.com/word/index.php?id=1658
#8 dziekuje   Thank(1)-     https://www.handspeak.com/word/index.php?id=2186
#9 gdzie      Where(1 NIE)- https://www.handspeak.com/word/index.php?id=2391
#10 jest      is(1)-        https://www.handspeak.com/word/index.php?id=10338
#11 bolec     Hurt(2)-      https://www.handspeak.com/word/index.php?id=1077
#12 dusic sie Choke(1 NIE)- https://www.handspeak.com/word/index.php?id=5584
#13 spragniony Thirsty(1)-  https://www.handspeak.com/word/index.php?id=2205
#14 glodny     hungry(1)-   https://www.handspeak.com/word/index.php?id=1076
#14.5 jesc     eat(1 NIE)-  https://www.handspeak.com/word/index.php?id=645
#15 zmarzniety cold(2)-     https://www.handspeak.com/word/index.php?id=645
#16 resustytacja RESUSCITATION(2) https://www.handspeak.com/word/index.php?id=5154
#17 zemdlal    blackout (1) https://www.handspeak.com/word/index.php?id=5618
!#18 cukrzyk    diabetic (1) https://www.handspeak.com/word/index.php?id=1092
#19 zlamal :)   broke (1)-  https://www.handspeak.com/word/index.php?id=263
#20 reka        arm(1)    https://www.handspeak.com/word/index.php?id=105
#21 noga!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#22 uderzyc    hit(2)- https://www.handspeak.com/word/index.php?id=1037
#23 glowa
#24 krwawi
#25 wypadek
#26 karetka
#27 zadzwon
#28 pic
#29 porazony pradem
#30 skelpie
#31 ulicy
#32 w
#33 na
#34 przy
#35 na pasach